In [3]:
# import library
import pandas as pd
import numpy as np

In [4]:
# read data as pandas dataframe
df = pd.read_csv('order_brush_order.csv')

In [16]:
df.head()

,orderid,shopid,userid
event_time,,,
2019-12-27 00:00:00,31075200506751,6042309,97707522
2019-12-27 00:00:00,31075200506752,104804492,97707522
2019-12-27 00:00:00,31075200506753,8715449,97707522
2019-12-27 00:00:02,31075201870570,190969466,170182475
2019-12-27 00:00:05,31075205798264,2859407,12532131


In [5]:
# change event_time columns to datetime data type
df['event_time'] = pd.to_datetime(df['event_time'])

In [6]:
# sort by event_time
df = df.sort_values(by='event_time').reset_index(drop=True)

In [7]:
# set event_time columns as index
df = df.set_index('event_time')

In [8]:
# using rolling function
df_grouped = df.groupby(['shopid', 'userid']).rolling('1H').orderid.count()
df_1h_count = pd.DataFrame(df_grouped).sort_values(by='shopid')

In [17]:
df_1h_count

,shopid,userid,event_time,orderid
0,10009,196962305,2019-12-27 03:06:50,1.0
1,10051,2854032,2019-12-27 19:16:11,1.0
2,10051,48600461,2019-12-29 01:56:19,1.0
3,10061,62464559,2019-12-31 02:58:48,1.0
4,10061,130633421,2019-12-30 21:35:15,1.0
...,...,...,...,...
222744,214949521,46269178,2019-12-31 20:06:43,2.0
222746,214964814,200983383,2019-12-29 22:26:16,1.0
222747,215175775,129266028,2019-12-31 14:14:37,1.0
222748,215175775,13688804,2019-12-31 09:06:31,1.0


In [9]:
# reset index
df_1h_count = df_1h_count.reset_index()

In [10]:
# filter base on maximum orderid per unique shopid and userid
order_max = df_1h_count.groupby(['shopid', 'userid']).orderid.transform(max)
df_1h_count = df_1h_count.loc[df_1h_count.orderid == order_max]

In [19]:
df_1h_count

,shopid,userid,event_time,orderid
0,10009,196962305,2019-12-27 03:06:50,1.0
1,10051,2854032,2019-12-27 19:16:11,1.0
2,10051,48600461,2019-12-29 01:56:19,1.0
3,10061,62464559,2019-12-31 02:58:48,1.0
4,10061,130633421,2019-12-30 21:35:15,1.0
...,...,...,...,...
222744,214949521,46269178,2019-12-31 20:06:43,2.0
222746,214964814,200983383,2019-12-29 22:26:16,1.0
222747,215175775,129266028,2019-12-31 14:14:37,1.0
222748,215175775,13688804,2019-12-31 09:06:31,1.0


In [11]:
# divided dataframe into 2 category (sub_yes = score >=3, sub_no = score< 3)
sub_yes = df_1h_count[df_1h_count['orderid'] >= 3.0].reset_index(drop=True)
sub_no = df_1h_count[df_1h_count['orderid'] < 3.0].reset_index(drop=True)

In [21]:
display(sub_no), display(sub_no)

,shopid,userid
0,10009,0
1,10051,0
2,10051,0
3,10061,0
4,10061,0
...,...,...
212235,214949521,0
212236,214964814,0
212237,215175775,0
212238,215175775,0


,shopid,userid
0,10009,0
1,10051,0
2,10051,0
3,10061,0
4,10061,0
...,...,...
212235,214949521,0
212236,214964814,0
212237,215175775,0
212238,215175775,0


(None, None)

In [12]:
# change data type to string
sub_yes['shopid'] = sub_yes['shopid'].astype(str)
sub_yes['userid'] = sub_yes['userid'].astype(str)
# change data type to string
sub_no['shopid'] = sub_no['shopid'].astype(str)
sub_no['userid'] = sub_no['userid'].astype(str)

In [13]:
# join by & for every 'userid' that has same 'shopid'
sub_yes = sub_yes.groupby('shopid')['userid'].apply('&'.join).reset_index()

In [14]:
# change userid to 0 for sub_no dataframe (score < 3)
sub_no['userid'] = '0'
sub_no = sub_no[['shopid','userid']]

In [15]:
# concat sub_yes and sub_no dataframe and drop duplicated shopid
df_new = pd.concat([sub_yes, sub_no], axis=0).reset_index(drop=True)
df_new = df_new.drop_duplicates(subset='shopid', keep='first').reset_index(drop=True)

In [22]:
df_new

,shopid,userid
0,100446829,2434757
1,10159,214988798
2,10199219,8405753
3,10206302,95058664
4,102111330,59440029
...,...,...
18765,214662358,0
18766,214949521,0
18767,214964814,0
18768,215175775,0
